## **Fine-tuning BERT for named-entity recognition**

In this notebook, we are going to use **BertForTokenClassification** which is included in the [Transformers library](https://github.com/huggingface/transformers) by HuggingFace. This model has BERT as its base architecture, with a token classification head on top, allowing it to make predictions at the token level, rather than the sequence level. Named entity recognition is typically treated as a token classification problem, so that's what we are going to use it for.

This tutorial uses the idea of **transfer learning**, i.e. first pretraining a large neural network in an unsupervised way, and then fine-tuning that neural network on a task of interest. In this case, BERT is a neural network pretrained on 2 tasks: masked language modeling and next sentence prediction. Now, we are going to fine-tune this network on a NER dataset. Fine-tuning is supervised learning, so this means we will need a labeled dataset.

Now, let's move on to the real stuff!

#### **Importing Python Libraries and preparing the environment**

This notebook assumes that you have the following libraries installed:
* pandas
* numpy
* sklearn
* pytorch
* transformers
* seqeval

As we are running this in Google Colab, the only libraries we need to additionally install are transformers and seqeval (GPU version):

In [ ]:
!pip install transformers seqeval[gpu]

     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 77 kB 5.6 MB/s 
     |████████████████████████████████| 895 kB 37.6 MB/s 
     |████████████████████████████████| 596 kB 5.3 MB/s 
     |████████████████████████████████| 6.6 MB 35.9 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=f3aec5335189761505e96a7829934bc225c448b176e10eee3c4cfe89a5196448
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification

As deep learning can be accellerated a lot using a GPU instead of a CPU, make sure you can run this notebook in a GPU runtime (which Google Colab provides for free! - check "Runtime" - "Change runtime type" - and set the hardware accelerator to "GPU").

We can set the default device to GPU using the following code (if it prints "cuda", it means the GPU has been recognized):

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


#### **Downloading and preprocessing the data**
Named entity recognition (NER) uses a specific annotation scheme, which is defined (at least for European languages) at the *word* level. An annotation scheme that is widely used is called **[IOB-tagging](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)**, which stands for Inside-Outside-Beginning. Each tag indicates whether the corresponding word is *inside*, *outside* or at the *beginning* of a specific named entity. The reason this is used is because named entities usually comprise more than 1 word. 

Let's have a look at an example. If you have a sentence like "Barack Obama was born in Hawaï", then the corresponding tags would be   [B-PERS, I-PERS, O, O, O, B-GEO]. B-PERS means that the word "Barack" is the beginning of a person, I-PERS means that the word "Obama" is inside a person, "O" means that the word "was" is outside a named entity, and so on. So one typically has as many tags as there are words in a sentence.

So if you want to train a deep learning model for NER, it requires that you have your data in this IOB format (or similar formats such as [BILOU](https://stackoverflow.com/questions/17116446/what-do-the-bilou-tags-mean-in-named-entity-recognition)). There exist many annotation tools which let you create these kind of annotations automatically (such as Spacy's [Prodigy](https://prodi.gy/), [Tagtog](https://docs.tagtog.net/) or [Doccano](https://github.com/doccano/doccano)). You can also use Spacy's [biluo_tags_from_offsets](https://spacy.io/api/goldparse#biluo_tags_from_offsets) function to convert annotations at the character level to IOB format.

Here, we will use a NER dataset from [Kaggle](https://www.kaggle.com/namanj27/ner-dataset) that is already in IOB format. One has to go to this web page, download the dataset, unzip it, and upload the csv file to this notebook. Let's print out the first few rows of this csv file:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/FL/NER/train.txt', 'r') as trainFile:
  datafile1 = trainFile.readlines()

labels_unique=[]
sentences = []
word_labels = []
sub_sentences = []
sub_word_labels=[]
for line in datafile1:
  if line!='\n':
    word, label = line.split('\t')
    sub_sentences.append(word)
    sub_word_labels.append(label.strip())
    if label.strip() not in labels_unique:
      labels_unique.append(label.strip())
  else:
    sentences.append(sub_sentences)
    word_labels.append(sub_word_labels)
    sub_sentences = []
    sub_word_labels=[]


print(len(sentences))
print(len(word_labels))
print(sentences)
print(word_labels)
print(labels_unique)

2810
2810
[['Super', 'Mario', 'Run', 'Malware', '#', '2', '–', 'DroidJack', 'RAT', 'Gamers', 'love', 'Mario', 'and', 'Pokemon', ',', 'but', 'so', 'do', 'malware', 'authors', '.'], ['A', 'few', 'days', 'back', ',', 'we', 'wrote', 'about', 'an', 'Android', 'Marcher', 'trojan', 'variant', 'posing', 'as', 'the', 'Super', 'Mario', 'Run', 'game', 'for', 'Android', '.'], ['We', 'have', 'found', 'another', 'instance', 'of', 'malware', 'posing', 'as', 'the', 'Super', 'Mario', 'Run', 'Android', 'app', ',', 'and', 'this', 'time', 'it', 'has', 'taken', 'the', 'form', 'of', 'DroidJack', 'RAT', '(', 'remote', 'access', 'trojan', ')', '.'], ['Proofpoint', 'wrote', 'about', 'the', 'DroidJack', 'RAT', 'side-loaded', 'with', 'the', 'Pokemon', 'GO', 'app', 'back', 'in', 'July', '2016', ';', 'the', 'difference', 'here', 'is', 'that', 'there', 'is', 'no', 'game', 'included', 'in', 'the', 'malicious', 'package', '.'], ['The', 'authors', 'are', 'trying', 'to', 'latch', 'onto', 'the', 'popularity', 'of', 'the

# New Section

In [ ]:
data_train = pd.DataFrame(columns=['sentence', 'word_labels'])
for i in range(len(sentences)):
  sentence = " ".join(sentences[i])
  labels = ",".join(word_labels[i])
  data_train.loc[len(data_train.index)] = [sentence, labels] 

In [ ]:
data_train = data_train[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
data_train.head()

,sentence,word_labels
0,Super Mario Run Malware # 2 – DroidJack RAT Ga...,"B-Malware,I-Malware,I-Malware,I-Malware,O,O,O,..."
1,"A few days back , we wrote about an Android Ma...","O,O,O,O,O,O,O,O,O,B-System,B-Malware,O,O,O,O,O..."
2,We have found another instance of malware posi...,"O,O,O,O,O,O,O,O,O,O,B-System,I-System,I-System..."
3,Proofpoint wrote about the DroidJack RAT side-...,"B-Organization,O,O,O,B-Malware,I-Malware,O,O,O..."
4,The authors are trying to latch onto the popul...,"O,O,O,O,O,O,O,O,O,O,O,B-System,I-System,I-Syst..."


We create 2 dictionaries: one that maps individual tags to indices, and one that maps indices to their individual tags. This is necessary in order to create the labels (as computers work with numbers = indices, rather than words = tags) - see further in this notebook.

In [ ]:
label2id = {k: v for v, k in enumerate(labels_unique)}
id2label = {v: k for v, k in enumerate(labels_unique)}
label2id

{'B-Indicator': 6,
 'B-Malware': 0,
 'B-Organization': 5,
 'B-System': 3,
 'B-Vulnerability': 9,
 'I-Indicator': 8,
 'I-Malware': 1,
 'I-Organization': 7,
 'I-System': 4,
 'I-Vulnerability': 10,
 'O': 2}

In [ ]:
with open('/content/drive/MyDrive/FL/NER/valid.txt', 'r') as validFile:
  datafile2 = validFile.readlines()

labels_unique=[]
sentences = []
word_labels = []
sub_sentences = []
sub_word_labels=[]
for line in datafile2:
  if line!='\n':
    word, label = line.split('\t')
    sub_sentences.append(word)
    sub_word_labels.append(label.strip())
    if label.strip() not in labels_unique:
      labels_unique.append(label.strip())
  else:
    sentences.append(sub_sentences)
    word_labels.append(sub_word_labels)
    sub_sentences = []
    sub_word_labels=[]


print(len(sentences))
print(len(word_labels))
print(sentences)
print(word_labels)
print(labels_unique)

812
812
[['Riltok', 'mobile', 'Trojan', ':', 'A', 'banker', 'with', 'global', 'reach', '25', 'JUN', '2019', 'Riltok', 'is', 'one', 'of', 'numerous', 'families', 'of', 'mobile', 'banking', 'Trojans', 'with', 'standard', '(', 'for', 'such', 'malware', ')', 'functions', 'and', 'distribution', 'methods', '.'], ['Originally', 'intended', 'to', 'target', 'the', 'Russian', 'audience', ',', 'the', 'banker', 'was', 'later', 'adapted', ',', 'with', 'minimal', 'modifications', ',', 'for', 'the', 'European', '“', 'market.', '”', 'The', 'bulk', 'of', 'its', 'victims', '(', 'more', 'than', '90', '%', ')', 'reside', 'in', 'Russia', ',', 'with', 'France', 'in', 'second', 'place', '(', '4', '%', ')', '.'], ['Third', 'place', 'is', 'shared', 'by', 'Italy', ',', 'Ukraine', ',', 'and', 'the', 'United', 'Kingdom', '.'], ['We', 'first', 'detected', 'members', 'of', 'this', 'family', 'back', 'in', 'March', '2018', '.'], ['Like', 'many', 'other', 'bankers', ',', 'they', 'were', 'disguised', 'as', 'apps', 'for

In [ ]:
data_val = pd.DataFrame(columns=['sentence', 'word_labels'])
for i in range(len(sentences)):
  sentence = " ".join(sentences[i])
  labels = ",".join(word_labels[i])
  data_val.loc[len(data_val.index)] = [sentence, labels] 

In [ ]:
data_val = data_val[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
data_val.head()

,sentence,word_labels
0,Riltok mobile Trojan : A banker with global re...,"B-Malware,O,O,O,O,O,O,O,O,O,O,O,B-Malware,O,O,..."
1,Originally intended to target the Russian audi...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
2,"Third place is shared by Italy , Ukraine , and...","O,O,O,O,O,O,O,O,O,O,O,O,O,O"
3,We first detected members of this family back ...,"O,O,O,O,O,O,O,O,O,O,O,O"
4,"Like many other bankers , they were disguised ...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"


In [ ]:
with open('/content/drive/MyDrive/FL/NER/test.txt', 'r') as testFile:
  datafile3 = testFile.readlines()

labels_unique=[]
sentences = []
word_labels = []
sub_sentences = []
sub_word_labels=[]
for line in datafile3:
  if line!='\n':
    word, label = line.split('\t')
    sub_sentences.append(word)
    sub_word_labels.append(label.strip())
    if label.strip() not in labels_unique:
      labels_unique.append(label.strip())
  else:
    sentences.append(sub_sentences)
    word_labels.append(sub_word_labels)
    sub_sentences = []
    sub_word_labels=[]


print(len(sentences))
print(len(word_labels))
print(sentences)
print(word_labels)
print(labels_unique)

747
747
[['Why', 'Did', 'Chinese', 'Spyware', 'Linger', 'in', 'U.S', '.'], ['Phones', '?'], ['November', '16', ',', '2016', 'In', 'what', "'s", 'being', 'chalked', 'up', 'as', 'an', 'apparent', 'mistake', ',', 'more', 'than', '120,000', 'Android', 'phones', 'sold', 'in', 'the', 'U.S.', 'were', 'shipped', 'with', 'spying', 'code', 'that', 'sent', 'text', 'messages', ',', 'call', 'logs', 'and', 'other', 'sensitive', 'data', 'to', 'a', 'server', 'in', 'Shanghai', '.'], ['The', 'New', 'York', 'Times', 'reported', 'on', 'Nov.', '15', 'that', 'Kryptowire', ',', 'a', 'mobile', 'enterprise', 'security', 'company', ',', 'discovered', 'the', 'code', 'on', 'a', 'lower-end', 'smartphone', 'made', 'by', 'BLU', 'Products', 'of', 'Doral', ',', 'Fla', '.'], ['The', 'phones', 'are', 'sold', 'at', 'Best', 'Buy', 'and', 'Amazon.com', ',', 'among', 'other', 'retail', 'outlets', '.'], ['Kryptowire', 'says', 'the', 'code', ',', 'which', 'it', 'found', 'on', 'a', 'BLU', 'R1', 'HD', 'devices', ',', 'transmitt

In [ ]:
data_test = pd.DataFrame(columns=['sentence', 'word_labels'])
for i in range(len(sentences)):
  sentence = " ".join(sentences[i])
  labels = ",".join(word_labels[i])
  data_test.loc[len(data_test.index)] = [sentence, labels] 

In [ ]:
data_test = data_test[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
data_test.head()

,sentence,word_labels
0,Why Did Chinese Spyware Linger in U.S .,"O,O,O,O,O,O,O,O"
1,Phones ?,"O,O"
2,"November 16 , 2016 In what 's being chalked up...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-System,O..."
3,The New York Times reported on Nov. 15 that Kr...,"O,B-Organization,I-Organization,I-Organization..."
4,The phones are sold at Best Buy and Amazon.com...,"O,O,O,O,O,B-Organization,I-Organization,O,B-Or..."


In [ ]:
print(len(data_train))
print(len(data_test))
print(len(data_val))

2746
746
809


Let's verify that a random sentence and its corresponding tags are correct:

In [ ]:
data_train.iloc[41].sentence

'Technical analysis Most of this new attack ’ s routines are similar to those of the previous XLoader versions .'

In [ ]:
data_train.iloc[41].word_labels

'O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-Malware,O,O'

In [ ]:
data_test.iloc[41].sentence

'Skygofree : Following in the footsteps of HackingTeam 16 JAN 2018 At the beginning of October 2017 , we discovered new Android spyware with several features previously unseen in the wild .'

In [ ]:
data_test.iloc[41].word_labels

'B-Malware,O,O,O,O,O,O,B-Organization,O,O,O,O,O,O,O,O,O,O,O,O,O,B-System,O,O,O,O,O,O,O,O,O,O'

In [ ]:
data_val.iloc[41].sentence

'Sample configuration file of the Trojan Through AccessibilityService , the malware monitors AccessibilityEvent events .'

In [ ]:
data_val.iloc[41].word_labels

'O,O,O,O,O,O,O,O,O,O,O,O,O,O,O'

#### **Preparing the dataset and dataloader**

Now that our data is preprocessed, we can turn it into PyTorch tensors such that we can provide it to the model. Let's start by defining some key variables that will be used later on in the training/evaluation process:

In [ ]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """

    tokenized_sentence = []
    labels = []

    sentence = sentence.strip()

    for word, label in zip(sentence.split(), text_labels.split(",")):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

A tricky part of NER with BERT is that BERT relies on **wordpiece tokenization**, rather than word tokenization. This means that we should also define the labels at the wordpiece-level, rather than the word-level! 

For example, if you have word like "Washington" which is labeled as "b-gpe", but it gets tokenized to "Wash", "##ing", "##ton", then one approach could be to handle this by only train the model on the tag labels for the first word piece token of a word (i.e. only label "Wash" with "b-gpe"). This is what was done in the original BERT paper, see Github discussion [here](https://github.com/huggingface/transformers/issues/64#issuecomment-443703063).

Note that this is a **design decision**. You could also decide to propagate the original label of the word to all of its word pieces and let the model train on this. In that case, the model should be able to produce the correct labels for each individual wordpiece. This was done in [this NER tutorial with BERT](https://github.com/chambliss/Multilingual_NER/blob/master/python/utils/main_utils.py#L118). Another design decision could be to give the first wordpiece of each word the original word label, and then use the label “X” for all subsequent subwords of that word. All of them seem to lead to good performance.

Below, we define a regular PyTorch [dataset class](https://pytorch.org/docs/stable/data.html) (which transforms examples of a dataframe to PyTorch tensors). Here, each sentence gets tokenized, the special tokens that BERT expects are added, the tokens are padded or truncated based on the max length of the model, the attention mask is created and the labels are created based on the dictionary which we defined above. Word pieces that should be ignored have a label of -100 (which is the default `ignore_index` of PyTorch's [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)).

For more information about BERT's inputs, see [here](https://huggingface.co/transformers/glossary.html). 








In [ ]:
class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        # step 1: tokenize (and adapt corresponding labels)
        sentence = self.data.sentence[index]  
        word_labels = self.data.word_labels[index]  
        tokenized_sentence, labels = tokenize_and_preserve_labels(sentence, word_labels, self.tokenizer)
        
        # step 2: add special tokens (and corresponding labels)
        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"] # add special tokens
        labels.insert(0, "O") # add outside label for [CLS] token
        labels.insert(-1, "O") # add outside label for [SEP] token

        # step 3: truncating/padding
        maxlen = self.max_len

        if (len(tokenized_sentence) > maxlen):
          # truncate
          tokenized_sentence = tokenized_sentence[:maxlen]
          labels = labels[:maxlen]
        else:
          # pad
          tokenized_sentence = tokenized_sentence + ['[PAD]'for _ in range(maxlen - len(tokenized_sentence))]
          labels = labels + ["O" for _ in range(maxlen - len(labels))]

        # step 4: obtain the attention mask
        attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]
        
        # step 5: convert tokens to input ids
        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

        label_ids = [label2id[label] for label in labels]
        # the following line is deprecated
        #label_ids = [label if label != 0 else -100 for label in label_ids]
        
        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attn_mask, dtype=torch.long),
              #'token_type_ids': torch.tensor(token_ids, dtype=torch.long),
              'targets': torch.tensor(label_ids, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

Now, based on the class we defined above, we can create 2 datasets, one for training and one for testing. Let's use a 80/20 split:

In [ ]:
train_dataset = data_train
test_dataset = data_test
val_dataset = data_val
# train_dataset = train_dataset.reset_index(drop=True)

print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))
print("VALID Dataset: {}".format(val_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)
val_set = dataset(val_dataset, tokenizer, MAX_LEN)

TRAIN Dataset: (2746, 2)
TEST Dataset: (746, 2)
VALID Dataset: (809, 2)


Let's have a look at the first training example:

In [ ]:
training_set[0]

{'ids': tensor([  101,  3565,  7986,  2448, 15451,  8059,  1001,  1016,  1516,  2852,
          9314, 17364,  9350, 27911,  2015,  2293,  7986,  1998, 20421,  1010,
          2021,  2061,  2079, 15451,  8059,  6048,  1012,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,  

In [ ]:
training_set[0]["ids"]

tensor([  101,  3565,  7986,  2448, 15451,  8059,  1001,  1016,  1516,  2852,
         9314, 17364,  9350, 27911,  2015,  2293,  7986,  1998, 20421,  1010,
         2021,  2061,  2079, 15451,  8059,  6048,  1012,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

Let's verify that the input ids and corresponding targets are correct:

In [ ]:
# print the first 30 tokens and corresponding labels
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["ids"][:30]), training_set[0]["targets"][:30]):
  print('{0:10}  {1}'.format(token, id2label[label.item()]))

[CLS]       O
super       B-Malware
mario       I-Malware
run         I-Malware
mal         I-Malware
##ware      I-Malware
#           O
2           O
–           O
dr          B-Malware
##oid       B-Malware
##jack      B-Malware
rat         I-Malware
gamer       O
##s         O
love        O
mario       B-System
and         O
pokemon     B-System
,           O
but         O
so          O
do          O
mal         O
##ware      O
authors     O
.           O
[SEP]       O
[PAD]       O
[PAD]       O


Now, let's define the corresponding PyTorch dataloaders:

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }
              
val_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
val_loader = DataLoader(val_set, **val_params)
testing_loader = DataLoader(testing_set, **test_params)

#### **Defining the model**

Here we define the model, BertForTokenClassification, and load it with the pretrained weights of "bert-base-uncased". The only thing we need to additionally specify is the number of labels (as this will determine the architecture of the classification head).

Note that only the base layers are initialized with the pretrained weights. The token classification head of top has just randomly initialized weights, which we will train, together with the pretrained weights, using our labelled dataset. This is also printed as a warning when you run the code cell below.

Then, we move the model to the GPU.

In [ ]:
model = BertForTokenClassification.from_pretrained('bert-base-uncased', 
                                                   num_labels=len(id2label),
                                                   id2label=id2label,
                                                   label2id=label2id)
model.to(device)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

#### **Training the model**

Before training the model, let's perform a sanity check, which I learned thanks to Andrej Karpathy's wonderful [cs231n course](http://cs231n.stanford.edu/) at Stanford (see also his [blog post about debugging neural networks](http://karpathy.github.io/2019/04/25/recipe/)). The initial loss of your model should be close to -ln(1/number of classes) = -ln(1/17) = 2.83. 

Why? Because we are using cross entropy loss. The cross entropy loss is defined as -ln(probability score of the model for the correct class). In the beginning, the weights are random, so the probability distribution for all of the classes for a given token will be uniform, meaning that the probability for the correct class will be near 1/17. The loss for a given token will thus be -ln(1/17). As PyTorch's [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) (which is used by `BertForTokenClassification`) uses *mean reduction* by default, it will compute the mean loss for each of the tokens in the sequence for which a label is provided. 

Let's verify this:



In [ ]:
ids = training_set[0]["ids"].unsqueeze(0)
mask = training_set[0]["mask"].unsqueeze(0)
targets = training_set[0]["targets"].unsqueeze(0)
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]
initial_loss

tensor(2.4932, device='cuda:0', grad_fn=<NllLossBackward0>)

This looks good. Let's also verify that the logits of the neural network have a shape of (batch_size, sequence_length, num_labels):

In [ ]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 128, 11])

Next, we define the optimizer. Here, we are just going to use Adam with a default learning rate. One can also decide to use more advanced ones such as AdamW (Adam with weight decay fix), which is [included](https://huggingface.co/transformers/main_classes/optimizer_schedules.html) in the Transformers repository, and a learning rate scheduler, but we are not going to do that here.

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

Now let's define a regular PyTorch training function. It is partly based on [a really good repository about multilingual NER](https://github.com/chambliss/Multilingual_NER/blob/master/python/utils/main_utils.py#L344).

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_preds.extend(predictions)
        tr_labels.extend(targets)
        
        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

And let's train the model!

In [ ]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 2.4101710319519043
Training loss per 100 training steps: 0.41281030454331696
Training loss per 100 training steps: 0.27743784127877424
Training loss per 100 training steps: 0.2174569844512696
Training loss per 100 training steps: 0.18566819346112726
Training loss per 100 training steps: 0.16281397528797686
Training loss per 100 training steps: 0.14610432824910233
Training loss epoch: 0.1369750457449804
Training accuracy epoch: 0.9056460172783454


#### **Evaluating the model**

Now that we've trained our model, we can evaluate its performance on the held-out test set (which is 20% of the data). Note that here, no gradient updates are performed, the model just outputs its logits. 

In [ ]:
def valid(model, val_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(val_loader):
            
            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)
            
            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(targets)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
    
    
    labels = [id2label[id.item()] for id in eval_labels]
    predictions = [id2label[id.item()] for id in eval_preds]

       
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

As we can see below, performance is quite good! Accuracy on the test test is > 93%.

In [ ]:
labels, predictions = valid(model, val_loader)

Validation loss per 100 evaluation steps: 0.028900286182761192
Validation loss per 100 evaluation steps: 0.054901202816728244
Validation loss per 100 evaluation steps: 0.053878651642870735
Validation loss per 100 evaluation steps: 0.05164451793221404
Validation loss per 100 evaluation steps: 0.050087470934027226
Validation Loss: 0.049763071513734755
Validation Accuracy: 0.9415222991586889


In [ ]:
def testing(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)
            
            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Testing loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(targets)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
    

    labels = [id2label[id.item()] for id in eval_labels]
    predictions = [id2label[id.item()] for id in eval_preds]

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Testing Loss: {eval_loss}")
    print(f"Testing Accuracy: {eval_accuracy}")

    return labels, predictions

In [ ]:
labels, predictions = testing(model, testing_loader)

Testing loss per 100 evaluation steps: 0.003310648025944829
Testing loss per 100 evaluation steps: 0.07664672729195013
Testing loss per 100 evaluation steps: 0.06861140556976347
Testing loss per 100 evaluation steps: 0.07435986264554653
Testing Loss: 0.07968515181950177
Testing Accuracy: 0.9271631468635867


However, the accuracy metric is misleading, as a lot of labels are "outside" (O), even after omitting predictions on the [PAD] tokens. What is important is looking at the precision, recall and f1-score of the individual tags. For this, we use the seqeval Python library: 

In [ ]:
from seqeval.metrics import classification_report

print(classification_report([labels], [predictions]))

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


               precision    recall  f1-score   support

    Indicator       0.66      0.82      0.73      2261
      Malware       0.45      0.77      0.57       603
 Organization       0.00      0.00      0.00       241
       System       0.33      0.30      0.31       399
Vulnerability       0.00      0.00      0.00        47

    micro avg       0.58      0.68      0.63      3551
    macro avg       0.29      0.38      0.32      3551
 weighted avg       0.53      0.68      0.60      3551



In [ ]:
from seqeval.metrics import classification_report

print(classification_report([labels], [predictions]))

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


               precision    recall  f1-score   support

    Indicator       0.66      0.82      0.73      2261
      Malware       0.45      0.77      0.57       603
 Organization       0.00      0.00      0.00       241
       System       0.33      0.30      0.31       399
Vulnerability       0.00      0.00      0.00        47

    micro avg       0.58      0.68      0.63      3551
    macro avg       0.29      0.38      0.32      3551
 weighted avg       0.53      0.68      0.60      3551



Performance already seems quite good, but note that we've only trained for 1 epoch. An optimal approach would be to perform evaluation on a validation set while training to improve generalization.

#### **Inference**

The fun part is when we can quickly test the model on new, unseen sentences. 
Here, we use the prediction of the **first word piece of every word** (which is how the model was trained). 

*In other words, the code below does not take into account when predictions of different word pieces that belong to the same word do not match.*

In [ ]:
sentence = "Technical analysis Most of this new attack ’ s routines are similar to those of the previous XLoader versions"

inputs = tokenizer(sentence, padding='max_length', truncation=True, max_length=MAX_LEN, return_tensors="pt")

# move to gpu
ids = inputs["input_ids"].to(device)
mask = inputs["attention_mask"].to(device)
# forward pass
outputs = model(ids, mask)
logits = outputs[0]

active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
token_predictions = [id2label[i] for i in flattened_predictions.cpu().numpy()]
wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

word_level_predictions = []
for pair in wp_preds:
  if (pair[0].startswith(" ##")) or (pair[0] in ['[CLS]', '[SEP]', '[PAD]']):
    # skip prediction
    continue
  else:
    word_level_predictions.append(pair[1])

# we join tokens, if they are not special ones
str_rep = " ".join([t[0] for t in wp_preds if t[0] not in ['[CLS]', '[SEP]', '[PAD]']]).replace(" ##", "")
print(str_rep)
print(word_level_predictions)

technical analysis most of this new attack ’ s routines are similar to those of the previous xloader versions
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Malware', 'B-Malware', 'B-Malware', 'O']


#### **Saving the model for future use**

Finally, let's save the vocabulary (.txt) file, model weights (.bin) and the model's configuration (.json) to a directory, so that both the tokenizer and model can be re-loaded using the `from_pretrained()` class method.


In [ ]:
import os

directory = "./model"

if not os.path.exists(directory):
    os.makedirs(directory)

# save vocabulary of the tokenizer
tokenizer.save_vocabulary(directory)
# save the model weights and its configuration file
model.save_pretrained(directory)
print('All files saved')
print('This tutorial is completed')

All files saved
This tutorial is completed
